In [1]:
import warnings
warnings.filterwarnings('ignore')
from category_encoders import LeaveOneOutEncoder,WOEEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
import math


In [45]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostClassifier, Pool
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tensorflow
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

### **5 Leave-one-out Encoding**
In order not to split the data into three different sets, we will use a leave-one-out encoding. This type of encoding calculates the target response mean for each variable **excluding** the current observation to reduce the effect of outliers. It further adds random noise to the target response to avoid overfitting.

In [3]:
df = pd.read_csv("df_processed.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-16,0.0,4,2012-04-02_825,48,411,0,4,4,4,2,6,14,14,3,249.7,83.23,1,2,2,1,2,1,1,2,1,2,1,2,2,1,1,2,1,2,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531165,50074,2013-04-29,2013-05-03,2342,s,terracotta,5,69.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,1,8,5,4,3,6,4,5,1,8,1,8,3,6,4,5,3,6,1,8,3,6
531166,50075,2013-04-29,2013-05-03,2505,s,terracotta,5,64.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,2,7,5,4,3,6,4,5,2,7,2,7,3,6,4,5,3,6,2,7,3,6
531167,50076,2013-04-28,2013-05-02,2470,l,white,5,79.90,85095,Mrs,1950-02-14,Berlin,2013-03-24,NaN,4,2013-04-28_85095,63,35,6,3,4,5,28,2,17,18,1,79.9,79.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
531168,50077,2013-04-28,2013-05-02,2452,m,white,5,59.90,91922,Mrs,1969-11-27,Bburg,2013-04-28,NaN,4,2013-04-28_91922,43,0,6,3,4,5,28,2,17,18,2,119.8,59.90,2,0,2,0,1,1,2,0,2,0,1,1,1,1,2,0,1,1,1,1,1,1


In [4]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols

In [27]:
# Get the real class predictions for the test set
# real_class = pd.read_csv("orders_realclass.txt", delimiter=";")
# real_class = real_class.astype({"orderItemID":int,"returnShipment":int})
# df_valid.reset_index(drop=True,inplace=True)
# df_valid["return"] = real_class["returnShipment"]

In [28]:
# Combine the training and former testing sets into one dataframe
# df_train_new = pd.concat([df_train,df_test],axis=0)
# # Encoding the categorical variables
# df_train_encoded, df_test_encoded, encoder = encode_categorical_columns(df_train_new,df_valid,cat_cols,0.1)

### **Additional Features**

In [5]:
# Get number of unique values in a column
list_include = ["item_id","size","order_id","brand_id","item_color"]
column_name= {"item_id":"order_item_id_nunique","size":"order_size_nunique","brand_id":"order_brand_id_nunique","item_color":"order_item_color_nunique"}
df_group = df[list_include].groupby(["order_id"]).agg({"item_id":"nunique","size":"nunique","brand_id":"nunique","item_color":"nunique"}).reset_index().rename(columns= column_name)
df = df.merge(df_group,on="order_id",how="left")

In [6]:
# Same item_id different colors, sizes
df_group = df[list_include].groupby(["order_id","item_id"]).agg({"item_color":"nunique","size":"nunique"}).reset_index().rename(columns= {"item_color":"order_item_id_color_nunique","size":"order_item_id_size_nunique"})
df = df.merge(df_group,on=["order_id","item_id"],how="left")

In [7]:
# Same brand_id different colors, sizes, item_id
df_group = df[list_include].groupby(["order_id","brand_id"]).agg({"item_color":"nunique","size":"nunique","item_id":"nunique"}).reset_index().rename(columns= {"item_color":"order_brand_id_color_nunique","size":"order_brand_id_size_nunique","item_id":"order_brand_id_item_id_nunique"})
df = df.merge(df_group,on=["order_id","brand_id"],how="left")

In [8]:
stats_to_agg = ["mean","std","min","max","sum","count","median","mad"]
# Item id price statistics
df_group = df[["item_id","item_price"]].groupby(["item_id"]).agg(stats_to_agg).reset_index().round(2)
df_group.columns = ['_item_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"item_id_item_id_":"item_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="item_id",how="left")

In [9]:
# user_id price statistics
df_group = df[["user_id","item_price"]].groupby(["user_id"]).agg(stats_to_agg).reset_index().round(2)
# df_group.columns = ['_'.join(col) for col in df_group.columns]
df_group.columns = ['_user_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"user_id_user_id_":"user_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="user_id",how="left")

In [10]:
# brand_id price statistics
df_group = df[["brand_id","item_price"]].groupby(["brand_id"]).agg(stats_to_agg).reset_index().round(2)
df_group.columns = ['_brand_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"brand_id_brand_id_":"brand_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="brand_id",how="left")

In [13]:
# Get difference between the min, max and mean of the item_price column and the item_price column
min_max_mean = ["min","max","mean"]
for i in df.columns:
    for j in min_max_mean:
        if j in i:
            df["price-"+ i] = df[i] - df["item_price"] 

In [14]:
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [ ]:
df_group = df[["item_id","size","brand_id","item_color","user_id"]].groupby(["user_id"]).agg(lambda x: pd.Series.mode(x)[0])
df_group = df_group.add_prefix("mode_")
df = df.merge(df_group,on="user_id",how="left")

In [31]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,mode_item_id,mode_size,mode_brand_id,mode_item_color,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,71,s,20,white,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,31.10,8.68,19.90,39.90,68669.20,2208,29.90,8.41,39.90,0.00,39.9,39.9,39.9,1,39.9,0.00,55.16,19.34,0.0,89.90,629751.00,11417,49.90,16.21,22,s,14,green,-8.80,-20.0,0.0,0.00,0.00,0.00,15.26,-39.90,50.0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-

In [54]:
cols_to_agg = ["item_id","size","brand_id","item_color","user_id"]
stats_to_agg = ["mean","std","min","max","median","mad"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group = df[[i,"delivery_time"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
    df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group.fillna(0,inplace=True)
    df = df.merge(df_group,on=i,how="left")    

In [55]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,mode_item_id,mode_size,mode_brand_id,mode_item_color,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,71,s,20,white,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,5.04,5.80,1,30,3.0,3.61,10.38,16.45,0,71,4.0,11.27,4.21,5.15,0,71,3.0,2.65,9.80,15.10,0,71,4.0,10.38,3.64,3.38,1,13,3.0,2.20
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19

In [57]:
# Instantiate the model class
Model = Model_class(df)

In [58]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)
# Encode the categorical variables using leave-one-out encoding without dropping the categorical columns
df_train_encoded_whole,df_test_encoded_whole,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,False)

In [18]:
catboost_params = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model, catboost_Y_pred, catboost_mae, catboost_Y_pred_proba = Model.catboost(df_train,df_test,catboost_params,1,cat_cols)
print("Catboost MAE on test set:", catboost_mae)

0:	learn: 0.6779730	test: 0.6801536	best: 0.6801536 (0)	total: 937ms	remaining: 2m 19s
1:	learn: 0.6657492	test: 0.6691639	best: 0.6691639 (1)	total: 1.89s	remaining: 2m 20s
2:	learn: 0.6560998	test: 0.6600287	best: 0.6600287 (2)	total: 2.74s	remaining: 2m 14s
3:	learn: 0.6441531	test: 0.6519582	best: 0.6519582 (3)	total: 3.42s	remaining: 2m 4s
4:	learn: 0.6339603	test: 0.6448618	best: 0.6448618 (4)	total: 4.05s	remaining: 1m 57s
5:	learn: 0.6262934	test: 0.6396703	best: 0.6396703 (5)	total: 4.66s	remaining: 1m 51s
6:	learn: 0.6196555	test: 0.6349094	best: 0.6349094 (6)	total: 5.45s	remaining: 1m 51s
7:	learn: 0.6131913	test: 0.6301271	best: 0.6301271 (7)	total: 6.09s	remaining: 1m 48s
8:	learn: 0.6078442	test: 0.6258750	best: 0.6258750 (8)	total: 6.73s	remaining: 1m 45s
9:	learn: 0.6034258	test: 0.6225074	best: 0.6225074 (9)	total: 7.36s	remaining: 1m 43s
10:	learn: 0.6004448	test: 0.6205561	best: 0.6205561 (10)	total: 7.98s	remaining: 1m 40s
11:	learn: 0.5977594	test: 0.6185539	best:

In [47]:
Model.get_feature_importance(catboost_model,df_train)

[('delivery_time', 15.655995287477024),
 ('user_id', 13.729875442954917),
 ('delivery_weekday', 8.11639207135039),
 ('item_id', 7.98920008977104),
 ('order_sum', 4.30900885146087),
 ('order_item_id_size_nunique', 2.0619282259540457),
 ('item_price_user_id_sum', 1.7454182862984993),
 ('brand_id', 1.6629908750030005),
 ('user_reg_age', 1.4009884844594236),
 ('price-item_price_item_id_mean', 1.3656404019562416),
 ('item_price_user_id_count', 1.2978413456872948),
 ('item_color', 1.2865680221827285),
 ('user_age', 1.2614802212944862),
 ('item_price', 1.2594152839589563),
 ('order_number_same_item_id_item_color', 1.1173437188210025),
 ('delivery_month', 1.0545525087603884),
 ('order_weekday', 1.047765911054712),
 ('delivery_week', 1.0475265995719154),
 ('size', 0.9198608450468988),
 ('mode_size', 0.9155261676160902),
 ('mode_brand_id', 0.9049703966923981),
 ('item_price_user_id_median', 0.8507547123466189),
 ('mode_item_id', 0.8405994744220966),
 ('order_item_count', 0.7962433524330313),
 ('

In [59]:
catboost_params = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model, catboost_Y_pred, catboost_mae, catboost_Y_pred_proba = Model.catboost(df_train,df_test,catboost_params,1,cat_cols)
print("Catboost MAE on test set:", catboost_mae)

0:	learn: 0.6779059	test: 0.6795504	best: 0.6795504 (0)	total: 815ms	remaining: 2m 1s
1:	learn: 0.6656576	test: 0.6687221	best: 0.6687221 (1)	total: 1.67s	remaining: 2m 3s
2:	learn: 0.6559701	test: 0.6601906	best: 0.6601906 (2)	total: 2.49s	remaining: 2m 2s
3:	learn: 0.6440463	test: 0.6519626	best: 0.6519626 (3)	total: 3.25s	remaining: 1m 58s
4:	learn: 0.6330480	test: 0.6436118	best: 0.6436118 (4)	total: 4.01s	remaining: 1m 56s
5:	learn: 0.6242483	test: 0.6375056	best: 0.6375056 (5)	total: 4.76s	remaining: 1m 54s
6:	learn: 0.6177960	test: 0.6329470	best: 0.6329470 (6)	total: 5.51s	remaining: 1m 52s
7:	learn: 0.6118289	test: 0.6286269	best: 0.6286269 (7)	total: 6.25s	remaining: 1m 50s
8:	learn: 0.6072784	test: 0.6252377	best: 0.6252377 (8)	total: 6.98s	remaining: 1m 49s
9:	learn: 0.6032977	test: 0.6224745	best: 0.6224745 (9)	total: 7.73s	remaining: 1m 48s
10:	learn: 0.5993645	test: 0.6194115	best: 0.6194115 (10)	total: 8.54s	remaining: 1m 47s
11:	learn: 0.5965059	test: 0.6175544	best: 0

In [60]:
Model.get_feature_importance(catboost_model,df_train)

[('delivery_time', 17.19151063817235),
 ('user_id', 13.058865545441632),
 ('delivery_weekday', 8.727475431484091),
 ('item_id', 7.16512763833561),
 ('order_sum', 3.496313758614527),
 ('order_item_id_size_nunique', 1.940389704700041),
 ('delivery_time_user_id_median', 1.7708780275511604),
 ('delivery_time_user_id_min', 1.668951272982231),
 ('brand_id', 1.6385683872863828),
 ('item_price_user_id_sum', 1.3648377543780637),
 ('price-item_price_item_id_mean', 1.1263331840917103),
 ('order_number_same_item_id_item_color', 1.077384913204549),
 ('user_age', 1.056791829732049),
 ('item_price', 1.041792691596024),
 ('order_item_count', 0.9837149308899031),
 ('delivery_time_user_id_mad', 0.972743041695574),
 ('item_price_item_id_max', 0.9495607019514166),
 ('delivery_time_item_id_median', 0.9462503135491973),
 ('user_reg_age', 0.9278217192166377),
 ('item_price_user_id_count', 0.907907230030288),
 ('delivery_month', 0.8786477720229515),
 ('item_color', 0.8582290595718446),
 ('delivery_time_item_i

In [19]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.48901
[1]	validation_0-mae:0.47991
[2]	validation_0-mae:0.47162
[3]	validation_0-mae:0.46490
[4]	validation_0-mae:0.45906
[5]	validation_0-mae:0.45363
[6]	validation_0-mae:0.44916
[7]	validation_0-mae:0.44514
[8]	validation_0-mae:0.44181
[9]	validation_0-mae:0.43894
[10]	validation_0-mae:0.43608
[11]	validation_0-mae:0.43378
[12]	validation_0-mae:0.43163
[13]	validation_0-mae:0.42997
[14]	validation_0-mae:0.42836
[15]	validation_0-mae:0.42725
[16]	validation_0-mae:0.42586
[17]	validation_0-mae:0.42482
[18]	validation_0-mae:0.42361
[19]	validation_0-mae:0.42298
[20]	validation_0-mae:0.42217
[21]	validation_0-mae:0.42100
[22]	validation_0-mae:0.42043
[23]	validation_0-mae:0.41970
[24]	validation_0-mae:0.41929
[25]	validation_0-mae:0.41892
[26]	validation_0-mae:0.41790
[27]	validation_0-mae:0.41769
[28]	validation_0-mae:0.41743
[29]	validation_0-mae:0.41697
[30]	validation_0-mae:0.41664
[31]	validation_0-mae:0.41636
[32]	validation_0-mae:0.41611
[33]	validation_0-ma

In [51]:
Model.get_feature_importance(xgb_model,df_train_encoded)

[('user_id_LOE_encoded', 0.12702422),
 ('order_item_count', 0.09382843),
 ('item_id_LOE_encoded', 0.073513635),
 ('delivery_weekday', 0.06396292),
 ('order_item_id_size_nunique', 0.024929821),
 ('delivery_time', 0.023592481),
 ('order_number_same_item_id_item_color', 0.021782022),
 ('item_price', 0.017904403),
 ('order_sum', 0.017493503),
 ('order_number_different_item_id_size_item_color', 0.0139852725),
 ('order_number_different_item_id_item_color', 0.012730905),
 ('order_number_same_item_color_brand_id', 0.012127591),
 ('item_price_item_id_max', 0.011897254),
 ('delivery_month', 0.011552992),
 ('order_number_different_item_color_brand_id', 0.011428309),
 ('order_number_different_size_item_color_brand_id', 0.011078828),
 ('order_number_same_brand_id', 0.010771832),
 ('order_number_different_item_id_size', 0.009690527),
 ('order_number_same_item_id_size', 0.009466751),
 ('delivery_week', 0.009344973),
 ('item_price_user_id_count', 0.009319165),
 ('order_number_same_size', 0.008575227),

In [64]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.48932
[1]	validation_0-mae:0.48020
[2]	validation_0-mae:0.47190
[3]	validation_0-mae:0.46512
[4]	validation_0-mae:0.45879
[5]	validation_0-mae:0.45390
[6]	validation_0-mae:0.44952
[7]	validation_0-mae:0.44528
[8]	validation_0-mae:0.44185
[9]	validation_0-mae:0.43884
[10]	validation_0-mae:0.43605
[11]	validation_0-mae:0.43352
[12]	validation_0-mae:0.43109
[13]	validation_0-mae:0.42945
[14]	validation_0-mae:0.42747
[15]	validation_0-mae:0.42573
[16]	validation_0-mae:0.42428
[17]	validation_0-mae:0.42282
[18]	validation_0-mae:0.42179
[19]	validation_0-mae:0.42082
[20]	validation_0-mae:0.41960
[21]	validation_0-mae:0.41906
[22]	validation_0-mae:0.41814
[23]	validation_0-mae:0.41772
[24]	validation_0-mae:0.41725
[25]	validation_0-mae:0.41661
[26]	validation_0-mae:0.41585
[27]	validation_0-mae:0.41536
[28]	validation_0-mae:0.41478
[29]	validation_0-mae:0.41443
[30]	validation_0-mae:0.41396
[31]	validation_0-mae:0.41363
[32]	validation_0-mae:0.41331
[33]	validation_0-ma

In [65]:
Model.get_feature_importance(xgb_model,df_train_encoded)

[('user_id_LOE_encoded', 0.11326385),
 ('order_item_count', 0.076854885),
 ('item_id_LOE_encoded', 0.061575595),
 ('delivery_weekday', 0.05225607),
 ('order_item_id_size_nunique', 0.02248803),
 ('delivery_time', 0.021312006),
 ('order_number_same_item_id_item_color', 0.018756377),
 ('order_sum', 0.016923591),
 ('item_price', 0.015856285),
 ('delivery_time_user_id_median', 0.014500228),
 ('order_number_different_item_id_size_item_color', 0.013960559),
 ('order_number_same_item_color_brand_id', 0.0126719605),
 ('delivery_time_item_id_median', 0.011881348),
 ('item_price_item_id_max', 0.011498805),
 ('order_number_different_item_id_item_color', 0.010803977),
 ('order_number_different_item_id_size', 0.010684958),
 ('delivery_month', 0.009958794),
 ('order_number_same_brand_id', 0.008781175),
 ('delivery_time_user_id_min', 0.008635133),
 ('item_price_user_id_count', 0.008603014),
 ('order_number_same_item_id_size', 0.00803838),
 ('order_number_different_size_item_color_brand_id', 0.00773551

In [26]:
lightgbm_params = { "learning_rate":0.15, "n_estimators":150, "objective":"binary","random_state":42, "colsample_bytree" : 0.8}
lightgbm_model, lightgbm_Y_pred, lightgbm_mae, lightgbm_Y_pred_prob = Model.lightgmb(df_train, df_test, lightgbm_params, 1,cat_cols)
print("Lightgbm MAE on test set:", lightgbm_mae)

Lightgbm MAE on test set: 0.331


In [61]:
Model.get_feature_importance(lightgbm_model,df_train)

[('user_id', 2171),
 ('item_id', 1073),
 ('mode_item_id', 700),
 ('delivery_time', 71),
 ('brand_id', 61),
 ('mode_brand_id', 49),
 ('delivery_weekday', 44),
 ('order_sum', 41),
 ('item_color', 25),
 ('item_price_user_id_sum', 25),
 ('item_price_user_id_count', 17),
 ('delivery_week', 15),
 ('order_item_id_size_nunique', 15),
 ('order_number_same_item_id_item_color', 13),
 ('item_price_user_id_std', 13),
 ('user_age', 12),
 ('mode_item_color', 12),
 ('user_reg_age', 11),
 ('item_price', 10),
 ('delivery_month', 10),
 ('item_price_item_id_max', 9),
 ('price-item_price_item_id_mean', 8),
 ('order_number_different_item_id_size', 7),
 ('order_weekday', 6),
 ('user_state', 5),
 ('order_week', 5),
 ('item_price_user_id_min', 5),
 ('price-item_price_item_id_min', 5),
 ('price-item_price_user_id_max', 5),
 ('price-item_price_brand_id_mean', 5),
 ('order_item_count', 4),
 ('order_number_same_item_color_brand_id', 4),
 ('order_item_id_color_nunique', 4),
 ('size', 3),
 ('order_number_same_size',

In [62]:
lightgbm_params = { "learning_rate":0.15, "n_estimators":150, "objective":"binary","random_state":42, "colsample_bytree" : 0.8}
lightgbm_model, lightgbm_Y_pred, lightgbm_mae, lightgbm_Y_pred_prob = Model.lightgmb(df_train, df_test, lightgbm_params, 1,cat_cols)
print("Lightgbm MAE on test set:", lightgbm_mae)

Lightgbm MAE on test set: 0.3306


In [ ]:
cols_to_agg = ["item_id","size","brand_id","item_color","user_id"]
stats_to_agg = ["mean","std","min","max","median","mad"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group = df[[i,"delivery_time"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
    df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group.fillna(0,inplace=True)
    df = df.merge(df_group,on=i,how="left")    

In [ ]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,mode_item_id,mode_size,mode_brand_id,mode_item_color,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,71,s,20,white,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,5.04,5.80,1,30,3.0,3.61,10.38,16.45,0,71,4.0,11.27,4.21,5.15,0,71,3.0,2.65,9.80,15.10,0,71,4.0,10.38,3.64,3.38,1,13,3.0,2.20
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19

In [104]:
# df to csv
df.to_csv("df_97_columns.csv",index=False)